In [7]:
import spatialdata
import spatialdata_plot
import napari
import napari_spatialdata

import sopa.segmentation
import sopa.io
from sopa._sdata import get_spatial_image

import matplotlib.pyplot as plt
import math
import xarray

In [4]:
path_to_image = "/Users/jnimoca/Jose_BI/P26_SOPA_seg/991_subset.ome.tif"
sdata = sopa.io.ome_tif(path_to_image, as_image=False)

INFO     `dims` is specified redundantly: found also inside `data`.                                                


In [5]:
sdata

SpatialData object
└── Images
      └── '991_subset': DataTree[cyx] (15, 8000, 8000), (15, 4000, 4000), (15, 2000, 2000), (15, 1000, 1000), (15, 500, 500)
with coordinate systems:
    ▸ 'pixels', with elements:
        991_subset (Images)

In [6]:
patches = sopa.segmentation.Patches2D(sdata, element_name="991_subset", patch_width=1200, patch_overlap=100)
patches.write()

[INFO] (sopa.patches.patches) 64 patches were saved in sdata['sopa_patches']


,geometry,bboxes,ilocs
0,"POLYGON ((1200 0, 1200 1200, 0 1200, 0 0, 1200...","[0, 0, 1200, 1200]","[0, 0]"
1,"POLYGON ((2300 0, 2300 1200, 1100 1200, 1100 0...","[1100, 0, 2300, 1200]","[1, 0]"
2,"POLYGON ((3400 0, 3400 1200, 2200 1200, 2200 0...","[2200, 0, 3400, 1200]","[2, 0]"
3,"POLYGON ((4500 0, 4500 1200, 3300 1200, 3300 0...","[3300, 0, 4500, 1200]","[3, 0]"
4,"POLYGON ((5600 0, 5600 1200, 4400 1200, 4400 0...","[4400, 0, 5600, 1200]","[4, 0]"
...,...,...,...
59,"POLYGON ((4500 7700, 4500 8900, 3300 8900, 330...","[3300, 7700, 4500, 8900]","[3, 7]"
60,"POLYGON ((5600 7700, 5600 8900, 4400 8900, 440...","[4400, 7700, 5600, 8900]","[4, 7]"
61,"POLYGON ((6700 7700, 6700 8900, 5500 8900, 550...","[5500, 7700, 6700, 8900]","[5, 7]"
62,"POLYGON ((7800 7700, 7800 8900, 6600 8900, 660...","[6600, 7700, 7800, 8900]","[6, 7]"


In [8]:
def calculate_quantile(sdata, key:str, channels:list, quantile_value:float=0.9, return_ArrayData:bool=False):

    image = sdata.images[key]['scale0'].image
    assert type(image) == xarray.core.dataarray.DataArray, 'Image is not a xarray DataArray'
    selected_channels_image = image.sel(c=channels)

    quantile_projection = selected_channels_image.chunk({'c': -1}).quantile(quantile_value, dim='c', keep_attrs=True, interpolation='linear')
    #sdata does not like x,y only, so we create a dummy channel dimension
    quantile_projection_expanded = quantile_projection.expand_dims(dim='c', axis=0)

    if return_ArrayData:
        return quantile_projection_expanded
    else:
        sdata.images[f'{key}_q{int(quantile_value*100)}_projection'] = spatialdata.models.Image2DModel.parse(data=quantile_projection_expanded)
        return sdata

In [13]:
def quantiles_nuclear_membrane(sdata, key:str, nuclear_channels:list, membrane_channels:list, nuclear_quantile:float=0.9, membrane_quantile:float=0.9):

    nuclear_ArrayData = calculate_quantile(sdata, key, nuclear_channels, nuclear_quantile, return_ArrayData=True)
    membrane_ArrayData = calculate_quantile(sdata, key, membrane_channels, membrane_quantile, return_ArrayData=True)
    concatenated_array = spatialdata.models.Image2DModel.parse(xarray.concat([nuclear_ArrayData, membrane_ArrayData], dim='c'))
    
    name = f'{key}_n{int(nuclear_quantile*100)}_m{int(membrane_quantile*100)}_proj'

    sdata.images[name] = concatenated_array
    sdata.images[name] = sdata.images[name].assign_coords(c=['Nuclei', 'Membranes'])
    return sdata

In [14]:
sdata = quantiles_nuclear_membrane(sdata, '991_subset', 
                                nuclear_channels=["DAPI_1", "DAPI_2"],
                                membrane_channels=['Vimentin', 'CD3e', 'panCK', 'CD8', 'COL1A1', 'CD20', 'CD68'],
                                nuclear_quantile=0.9,
                                membrane_quantile=0.85)

/opt/homebrew/Caskroom/mambaforge/base/envs/SOPA3/lib/python3.10/site-packages/spatialdata/_core/_elements.py:62: UserWarning: Key `991_subset_n90_m85_proj` already exists. Overwriting it in-memory.
  self._check_key(key, self.keys(), self._shared_keys)
/opt/homebrew/Caskroom/mambaforge/base/envs/SOPA3/lib/python3.10/site-packages/spatialdata/_core/_elements.py:62: UserWarning: Key `991_subset_n90_m85_proj` already exists. Overwriting it in-memory.
  self._check_key(key, self.keys(), self._shared_keys)


In [15]:
sdata

SpatialData object
├── Images
│     ├── '991_subset': DataTree[cyx] (15, 8000, 8000), (15, 4000, 4000), (15, 2000, 2000), (15, 1000, 1000), (15, 500, 500)
│     └── '991_subset_n90_m85_proj': DataArray[cyx] (2, 8000, 8000)
└── Shapes
      └── 'sopa_patches': GeoDataFrame shape: (64, 3) (2D shapes)
with coordinate systems:
    ▸ 'pixels', with elements:
        991_subset (Images), 991_subset_n90_m85_proj (Images), sopa_patches (Shapes)

In [16]:
sdata.images['991_subset_n90_m85_proj']

<xarray.DataArray 'image' (c: 2, y: 8000, x: 8000)> Size: 1GB
dask.array<concatenate, shape=(2, 8000, 8000), dtype=float64, chunksize=(1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates:
    quantile  (c) float64 16B 0.9 0.85
  * y         (y) float64 64kB 0.5 1.5 2.5 3.5 ... 7.998e+03 7.998e+03 8e+03
  * x         (x) float64 64kB 0.5 1.5 2.5 3.5 ... 7.998e+03 7.998e+03 8e+03
  * c         (c) <U9 72B 'Nuclei' 'Membranes'
Attributes:
    transform:  {'pixels': Identity }

In [17]:
method = sopa.segmentation.methods.cellpose_patch(diameter=25, channels=["Membranes", "Nuclei"], flow_threshold=0.6, cellprob_threshold=-6, model_type="cyto3")
segmentation = sopa.segmentation.StainingSegmentation(sdata, method, image_key='991_subset_n90_m85_proj', channels=["Membranes", "Nuclei"], min_area=250)
cellpose_temp_dir = "/Users/jnimoca/Jose_BI/P26_SOPA_seg/Error_help.zarr/.sopa_cache/cellpose"

In [18]:
segmentation.write_patches_cells(cellpose_temp_dir)

[WARNING] (sopa.segmentation.stainings) Running segmentation in a sequential manner. This is not recommended on large images because it can be extremely slow (see https://github.com/gustaveroussy/sopa/discussions/36 for more details)
Run all patches:   0%|          | 0/64 [00:02<?, ?it/s]


ValueError: Images of type float must be between -1 and 1.